In [11]:
import pandas as pd
import pdb
import numpy as np
import itertools
import re
from utils import *
from sm_utils import *

from snakemake.io import expand
import yaml

In [12]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [13]:
def get_de_cfg_entries(p_df, cfg_entry, how):
    """
    Get file names needed as output for DE or DU
    tests within analysis objects for
        - all pairwise genotype sets
        - all pairwise genotype sets by sex

    Parameters:
        how (str): {'du', 'de'}
    """

    if how == 'du':
        feats = ['tss', 'tes', 'ic', 'iso']
    else:
        feats = []

    files = []

    for a in p_df.analysis.unique().tolist():
        wc = {'analysis': a}
        temp = subset_df_on_wcs(wc, p_df)
        obs_col = 'genotype_alias_int'
        conds = temp[obs_col].unique().tolist()

        combos = [c for c in itertools.combinations(conds, 2)]
        obs_cond1 = [c[0] for c in combos]
        obs_cond2 = [c[1] for c in combos]

        files += expand(expand(cfg_entry,
          zip,
          obs_cond1=obs_cond1,
          obs_cond2=obs_cond2,
          allow_missing=True),
          obs_col=obs_col,
          feat=feats,
          analysis=a)

        # now get genotype comparisons for each sex
        for s in temp.sex.unique():
            wc['sex'] = s
            temp2 = subset_df_on_wcs(wc, temp)
            obs_col = 'genotype_sex'
            conds = temp2[obs_col].unique().tolist()

            combos = [c for c in itertools.combinations(conds, 2)]
            obs_cond1 = [c[0] for c in combos]
            obs_cond2 = [c[1] for c in combos]

            files += expand(expand(cfg_entry,
              zip,
              obs_cond1=obs_cond1,
              obs_cond2=obs_cond2,
              allow_missing=True),
              obs_col=obs_col,
              feat=feats,
              analysis=a)

    return files

# 24 feb

In [14]:
wc = {'analysis':'240202_grant', 'obs_cond1':'5xFAD_F',
      'obs_cond2':'B6J_F', 'obs_col':'genotype_sex'}

## 243101

In [4]:
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
geno_tsv = 'genotype_metadata.tsv'
an_meta_tsv = 'analysis_config.tsv'
auto_dedupe = True

df, p_df = parse_config_file_analysis(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       geno_tsv,
                       an_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [10]:
   
wc = {  'analysis':'240202_grant',
  'study':'ad003',

  'genotype':'5xFAD-WT',
  'sex':'F',
  'age':'4_months',
  'tissue':'HC',
  'cerberus_run':'1',
  'end_mode':'tss'}
p_df.loc[(p_df.cerberus_run=='1')&\
         (p_df.analysis=='240202_grant')]
p_df[['cerberus_run', 'genotype']].drop_duplicates()
# get_cfg_entries_analysis(wc,
#                         p_df,
#                         config['analysis']['cerberus']['agg']['ends'])
# expand(get_cfg_entries_analysis(wc,
#                         p_df,
#                         config['analysis']['cerberus']['agg']['ends'])[0],
#                         end_mode='tss')[0]

,cerberus_run,genotype
0,1,5xFAD-HEMI
6,2,5xFAD-WT


In [27]:
wc = {  'analysis':'240202_grant',
  'study':'ad003',

  'genotype':'5xFAD-HEMI',
  'sex':'F',
  'age':'4_months',
  'tissue':'HC',
  'cerberus_run':'1',
  'end_mode':'tss'}
get_prev_cerb_entry(wc, p_df,
                      config['analysis']['cerberus']['agg']['ics'],
                      config)

'ref/ca_tss.bed'

In [7]:
# wc = {'analysis': '240202_grant',
#        analysis=240202_grant

#   study=ad003
#   genotype=5xFAD-WT
#   sex=F
#   age=4_months
#   tissue=HC
#   cerberus_run=1}

lambda wc:get_final_cerb_entry(wc,
                     p_df,
                     config['analysis']['cerberus']['ca'])[0]

<function __main__.<lambda>(wc)>

In [26]:
# p_df.dtypes

In [21]:
p_df.cerberus_run.unique()

array(['1', '2'], dtype=object)

In [25]:
p_df.loc[p_df.cerberus_run=='1'].head(1)[['study', 'genotype', 'sex', 'age', 'tissue', 'cerberus_run']]
# p_df.loc[p_df.cerberus_run=='1', 'genotype'].unique().tolist()

,study,genotype,sex,age,tissue,cerberus_run
0,ad003,5xFAD-HEMI,F,4_months,HC,1


In [16]:
wc = {'analysis': '240202_grant',

  'study':'ad003',
  'genotype':'5xFAD-WT',
  'sex':'F',
  'age':'4_months',
  'tissue':'HC',
  'cerberus_run':'1'}
expand(get_cfg_entries_analysis(wc,
                        p_df,
                        config['analysis']['cerberus']['agg']['ends'])[0],
                        end_mode='tss')[0]

'analysis/240202_grant/cerberus/agg/ad003_5xFAD-WT_F_4_months_HC_2_tss.bed'

In [10]:
get_cfg_entries_analysis(wc,
                        p_df,
                        config['analysis']['cerberus']['agg']['ends'])[0]

'analysis/240202_grant/cerberus/agg/ad003_5xFAD-HEMI_F_4_months_HC_1_{end_mode}.bed'

In [9]:
lambda wc:expand(get_cfg_entries_analysis(wc,
                        p_df,
                        config['analysis']['cerberus']['agg']['ends'])[0],
                        end_mode='tss')[0]

<function __main__.<lambda>(wc)>

In [10]:
get_de_cfg_entries(p_df, config['analysis']['swan']['du'], how='du')

['analysis/hClu/swan/5xFAD-HEMI_vs_B6J_genotype_alias_int_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_B6J_genotype_alias_int_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_B6J_genotype_alias_int_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_B6J_genotype_alias_int_du_iso.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_alias_int_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_alias_int_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_alias_int_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_alias_int_du_iso.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_Clu-h2kbKI-HO_genotype_alias_int_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_Clu-h2kbKI-HO_genotype_alias_int_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_Clu-h2kbKI-HO_genotype_alias_int_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_Clu-h2kbKI-HO_genotype_alias_int_du_iso.tsv',
 'analysis/hClu/swan/B6J_vs

## 243101

In [3]:
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
geno_tsv = 'genotype_metadata.tsv'
an_meta_tsv = 'analysis_config.tsv'
auto_dedupe = True

df, p_df = parse_config_file_analysis(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       geno_tsv,
                       an_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
p_df.genotype_alias.unique()

array(['5xFAD-HEMI', 'B6J', '5xFAD-HEMI;Clu-h2kbKI-HO', 'Clu-h2kbKI-HO',
       'hTREM2KI-HET', 'hTREM2-R47HKI-HET', 'hMAPT-HO;hABKI-SweIb-HET',
       'hMAPT-HO'], dtype=object)

In [5]:
p_df.genotype_alias_int.unique()

array(['5xFAD-HEMI', 'B6J', '5xFAD-HEMI_Clu-h2kbKI-HO', 'Clu-h2kbKI-HO',
       'hTREM2KI-HET', 'hTREM2-R47HKI-HET', 'hMAPT-HO_hABKI-SweIb-HET',
       'hMAPT-HO'], dtype=object)

In [7]:
p_df.genotype_sex.unique()

array(['5xFAD-HEMI_F', 'B6J_F', 'B6J_M', '5xFAD-HEMI_Clu-h2kbKI-HO_F',
       'Clu-h2kbKI-HO_F', 'hTREM2KI-HET_M', 'hTREM2-R47HKI-HET_M',
       'hTREM2-R47HKI-HET_F', 'hTREM2KI-HET_F',
       'hMAPT-HO_hABKI-SweIb-HET_F', 'hMAPT-HO_F',
       'hMAPT-HO_hABKI-SweIb-HET_M', 'hMAPT-HO_M'], dtype=object)

In [6]:
get_de_cfg_entries(p_df, config['analysis']['swan']['du'], how='du')

['analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_genotype_du_iso.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_du_iso.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_Clu-h2kbKI-HO_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_Clu-h2kbKI-HO_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_Clu-h2kbKI-HO_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFAD-HEMI_vs_5xFAD-WT_Clu-h2kbKI-HO_genotype_du_iso.tsv',
 'analysis/hClu/swan/5xFAD-WT_vs_5xFAD-HEMI_Clu-h2kbKI-HO_genotype_du_tss.tsv',
 'analysis/

## 243001

In [3]:
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
geno_tsv = 'genotype_metadata.tsv'
auto_dedupe = True

df, p_df = parse_config_file(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       geno_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
len(p_df.index)

114

In [5]:
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
geno_tsv = 'genotype_metadata.tsv'
an_meta_tsv = 'analysis_config.tsv'
auto_dedupe = True

df, p_df = parse_config_file_analysis(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       geno_tsv,
                       an_meta_tsv,
                       auto_dedupe=auto_dedupe)

> /Users/fairliereese/Documents/programming/mortazavi_lab/bin/modelad_pipeline/proc_update_2/sm_utils.py(232)parse_config_file_analysis()
    230     i3 = len(p_df[['genotype', 'study']].drop_duplicates().index)
    231     import pdb; pdb.set_trace()
--> 232     if not (i==i2==i3):
    233         genotypes = list(set(genotypes)-\
    234                          set(df.genotype.unique().tolist()))



ipdb>  genotypes


['hMAPT-HO_hABKI-SweIb-HET', 'hMAPT-HO_hABKI-SweIb-WT', 'hABKI-Swe-WT', '5xFAD-HEMI_Clu-h2kbKI-HO', '5xFAD-WT', '5xFAD-HEMI', '5xFAD-WT_Clu-h2kbKI-HO', 'hTREM2KI-HET', 'hTREM2-R47HKI-HET', 'hTREM2KI-WT']


ipdb>  c


In [6]:
p_df[['genotype', 'study', 'analysis']].drop_duplicates().sort_values(by='analysis')

,genotype,study,analysis
24,hABKI-Swe-WT,ad006,hABKI_Swe_IB_HET_WT
66,hMAPT-HO_hABKI-SweIb-WT,ad008,hABKI_Swe_IB_HET_WT
67,hMAPT-HO_hABKI-SweIb-HET,ad008,hABKI_Swe_IB_HET_WT
0,5xFAD-HEMI,ad003,hClu
6,5xFAD-WT,ad003,hClu
36,5xFAD-HEMI_Clu-h2kbKI-HO,ad003,hClu
38,5xFAD-WT_Clu-h2kbKI-HO,ad003,hClu
12,hTREM2KI-WT,ad004,hTREM2
48,hTREM2KI-HET,ad004,hTREM2
50,hTREM2-R47HKI-HET,ad004,hTREM2


In [9]:
len(p_df[['genotype', 'study', 'analysis']].drop_duplicates().index)

7

In [10]:
len(df.index)

53

## 240129

In [3]:
configfile: 'config.yml'
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
an_meta_tsv = 'test_analysis_config.tsv'
auto_dedupe = True

df, p_df = parse_config_file_analysis(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       an_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
len(p_df.index)

13

In [5]:
p_df.columns

Index(['batch', 'platform', 'fname', 'basename', 'path', 'flowcell',
       'file_stem', 'chop_num', 'sample_temp', 'mouse_id', 'study', 'genotype',
       'sex', 'age', 'tissue', 'pseudochrom_needed', 'pseudochrom', 'sample',
       'biorep_num', 'dataset', 'source', 'genotype_sex', 'human_gene',
       'mouse_gene', 'locus_type', 'notes', 'analysis', 'cerberus_run'],
      dtype='object')

In [6]:
df.columns

Index(['batch', 'platform', 'fname', 'basename', 'path', 'flowcell',
       'file_stem', 'chop_num', 'sample_temp', 'mouse_id', 'study', 'genotype',
       'sex', 'age', 'tissue', 'pseudochrom_needed', 'pseudochrom', 'sample',
       'biorep_num', 'dataset', 'source', 'genotype_sex', 'cerberus_run'],
      dtype='object')

In [ ]:
wc = {'}

In [12]:
get_cfg_entries(wc, df, config['analysis']['cerberus']['gtf'])

NameError: name 'wc' is not defined

In [5]:
# get_de_cfg_entries(p_df, config['analysis']['swan']['du'], how='du')
# get_de_cfg_entries(p_df, config['analysis']['swan']['deg'], how='de'),
# get_de_cfg_entries(p_df, config['analysis']['swan']['det'], how='de')

['analysis/hClu/swan/5xFADHEMI_vs_C57B6J_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_C57B6J_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_C57B6J_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_C57B6J_genotype_du_iso.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_5xCLU-h2kbKI_HO_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_5xCLU-h2kbKI_HO_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_5xCLU-h2kbKI_HO_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_5xCLU-h2kbKI_HO_genotype_du_iso.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_CLU-h2kbKI_HO_genotype_du_tss.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_CLU-h2kbKI_HO_genotype_du_tes.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_CLU-h2kbKI_HO_genotype_du_ic.tsv',
 'analysis/hClu/swan/5xFADHEMI_vs_CLU-h2kbKI_HO_genotype_du_iso.tsv',
 'analysis/hClu/swan/C57B6J_vs_5xCLU-h2kbKI_HO_genotype_du_tss.tsv',
 'analysis/hClu/swan/C57B6J_vs_5xCLU-h2kbKI_HO_genotype_du_tes.tsv',
 'analysis/hClu/swan/C57B6J_vs_5xCLU-h2kbKI_H

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19'], dtype=object)